In [55]:
import pandas as pd
import numpy as np
import os

In [56]:
!ls

clean_meas.csv
clean_measurements.csv
clean_station.csv
clean_stations.csv
climate_analysis.ipynb
data_engineering.ipynb
database_engineering.ipynb
hawaii.sqlite
images
merged_hawaii.csv
README.md
Resources
sqlalchemy


In [57]:
clean_meas = pd.read_csv('clean_meas.csv')
len(clean_meas)

19550

In [58]:
clean_meas.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
clean_meas.to_csv('clean_measurements.csv')
len(clean_meas)

19550

In [59]:
clean_meas.head(5)

,id,station,date,tobs
0,0,USC00519397,2010-01-01,65
1,1,USC00519397,2010-01-02,63
2,2,USC00519397,2010-01-03,74
3,3,USC00519397,2010-01-04,76
4,4,USC00519397,2010-01-06,73


In [60]:
clean_stations = pd.read_csv('clean_stations.csv')
clean_stations.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
clean_stations.to_csv('clean_station.csv')
clean_stations.head(1)

,id,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0


In [61]:
import sqlalchemy
import datetime
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, DateTime

In [62]:
engine = create_engine('sqlite:///hawaii.sqlite')

In [63]:
conn = engine.connect()

In [64]:
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Numeric)
    tobs = Column(Float)
    
        
    def __repr__(self):
        return f"id={self.id}, station={self.station}"

In [65]:
class Station(Base):
    __tablename__ = 'station'
    
    id = Column(Integer, primary_key = True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [66]:
Base.metadata.create_all(engine)

In [67]:
stations_df = pd.read_csv('clean_station.csv')
meas_df = pd.read_csv('clean_measurements.csv')

In [68]:
stations_data = stations_df.to_dict(orient='records')
meas_data = meas_df.to_dict(orient='records')

In [69]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [70]:
table_stations = sqlalchemy.Table('station', metadata, autoload=True)
table_meas = sqlalchemy.Table('measurement', metadata, autoload=True)

In [78]:
table_meas

Table('measurement', MetaData(bind=Engine(sqlite:///hawaii.sqlite)), Column('id', INTEGER(), table=<measurement>, primary_key=True, nullable=False), Column('station', VARCHAR(), table=<measurement>), Column('date', DATETIME(), table=<measurement>), Column('tobs', FLOAT(), table=<measurement>), schema=None)

In [71]:
conn.execute(table_stations.delete())

In [72]:
conn.execute(table_meas.delete())

In [73]:
conn.execute(table_stations.insert(), stations_data)

In [74]:
conn.execute(table_meas.insert(), meas_data)

StatementError: (builtins.TypeError) SQLite DateTime type only accepts Python datetime and date objects as input. [SQL: 'INSERT INTO measurement (id, station, date, tobs) VALUES (?, ?, ?, ?)'] [parameters: [{'Unnamed: 0': 0, 'id': 0, 'station': 'USC00519397', 'date': '2010-01-01', 'tobs': 65}, {'Unnamed: 0': 1, 'id': 1, 'station': 'USC00519397', 'date': '2010-01-02', 'tobs': 63}, {'Unnamed: 0': 2, 'id': 2, 'station': 'USC00519397', 'date': '2010-01-03', 'tobs': 74}, {'Unnamed: 0': 3, 'id': 3, 'station': 'USC00519397', 'date': '2010-01-04', 'tobs': 76}, {'Unnamed: 0': 4, 'id': 4, 'station': 'USC00519397', 'date': '2010-01-06', 'tobs': 73}, {'Unnamed: 0': 5, 'id': 5, 'station': 'USC00519397', 'date': '2010-01-07', 'tobs': 70}, {'Unnamed: 0': 6, 'id': 6, 'station': 'USC00519397', 'date': '2010-01-08', 'tobs': 64}, {'Unnamed: 0': 7, 'id': 7, 'station': 'USC00519397', 'date': '2010-01-09', 'tobs': 68}  ... displaying 10 of 19550 total bound parameter sets ...  {'Unnamed: 0': 19548, 'id': 19

In [75]:
conn.execute("select * from measurement").fetchall()

[]

In [76]:
conn.execute("select * from station").fetchall()

[(0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (2, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (4, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (5, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (6, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (7, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (8, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]